## Step for make it work

Install Redis for your OS
* pip install redis
* pip install python-redis-rate-limit # https://pypi.org/project/python-redis-rate-limit/
* pip install sparkly https://www.scivision.co/python-windows-visual-c++-14-required/
* pip install sparkly[redis]
* Install http://joeferner.github.io/redis-commander/  
Then `redis-commander --redis-host=localhost --redis-port=6379` from the browser http://localhost:8081/

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")

In [18]:
import redis
r = redis.StrictRedis(host='localhost', port=6379, db=0)

In [4]:
from sparkly import SparklySession
spark = SparklySession()

In [42]:
# make some test data
columns = ['dogs', 'cats']
vals = [
    (2, 0),
    (0, 1),
    (4, 1)
]

# create DataFrame
df = spark.createDataFrame(vals, columns)

In [46]:
import optimus as Optimus

In [47]:
df.table()

dogs 1 (bigint),cats 2 (bigint),id 3 (bigint)
2,0,17179869184
0,1,42949672960
4,1,60129542144


In [44]:
from pyspark.sql import functions as F

df = df.withColumn("id", F.monotonically_increasing_id())

In [45]:
df.show()

+----+----+-----------+
|dogs|cats|         id|
+----+----+-----------+
|   2|   0|17179869184|
|   0|   1|42949672960|
|   4|   1|60129542144|
+----+----+-----------+



In [55]:
df.write_ext.redis(
    host='localhost',
    port=6379,
    key_by=['id'],
    exclude_key_columns=True,
    expire=24 * 60 * 60,
    #compression='gzip',
)

In [21]:
r.get("1")
r.delete("1")
import requests 
result = requests.get("https://jsonplaceholder.typicode.com/posts/1")
print(result.text)

{
  "userId": 1,
  "id": 1,
  "title": "sunt aut facere repellat provident occaecati excepturi optio reprehenderit",
  "body": "quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto"
}


In [20]:
from redis_rate_limit import RateLimit, TooManyRequests
try:
    with RateLimit(resource='local', client='localhost', max_requests=10):
        result =  '200 OK'
except TooManyRequests:
    result = '429 Too Many Requests'
print(result)

200 OK


## Celery
* pip instal redis
* pip install celery
* pip install -U "celery[redis]"
* Run worker from python https://gist.github.com/chenjianjx/53d8c2317f6023dc2fa0

In [56]:
from optimus.enrichment.worker import download

In [61]:
r = download.delay('http"\\mood.com.ve')
r.ready()

False

In [58]:
r.ready()

False